<center><h5> Data Science For Suppy Chain & Operations </h5></center>
<center><h3> Etude de cas d'évaluation </h3></center>
<center><h1> Clustering de magasins </h1></center>

*Remplir ici votre compréhension du contexte et des objectifs*

# I. Chargement et création du jeu de données

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

In [ ]:
#Ceci permet d'afficher 
pd.options.display.max_columns = 100
pd.options.display.max_rows  = 100

In [ ]:
mag = pd.read_csv("Path to magasins.csv", encoding = "latin", sep = ";", decimal = ',') #Le paramètre decimal définit comment est représenter la virgule des chiffres décimaux
ref = pd.read_csv("Path to references_ete.csv", encoding = "latin", sep = ";")
ventes = pd.read_csv("Path to ventes.csv", encoding = "latin", sep = ";")

## I.1. Exploration, nettoyage et préparation des données

Nous allons procéder à l'exploration des différents fichiers fournis et au nettoyage de chacun des fichiers

### I.1.1. Exploration des fichiers 

#### Fichier magasin

*Insérez vos conclusions ici*

Quel est le nombre de magasins présent dans le fichier ? De quelles données disposons-nous ?

...

Quelques analyses à titre d'exemple...
- La variable Réseau peut être supprimée
- Nous pouvons uniquement garder le champ "Magasin" comme ID pour faire des jointures avec les autres tables, le champ code magasin peut être supprimé
- Il y a des données manquantes sur certains champs : 
    - Dans la colonne Mall, les valeurs manquantes signifient que les magasins ne sont pas dans des Malls
    - Dans d'autres colonnes catégoriques (Environnement concurrentiel, Traffic, Géographie, etc.) certains champs sont manquants. Il parait que l'information n'a pas été renseignée. Deux options s'offrent à nous : 
        - Supprimer ces colonnes 
        - Les compléter avec la valeur la plus présente
        - ...

        

In [ ]:
# Explorer le dataset

Afficher le nombre unique de "Magasin"


In [ ]:
#...

In [ ]:
categorical_variables = ['Réseau', 'Pays magasin','Type boutique',
       'Climat', 'Environnement concurrentiel','Traffic', 'Clientèle', 'Saisonnalité', 
        'Géographie', 'Mall', 'Comportement d\'achat']

In [ ]:
numerical_variables = ['Surface magasin (*)', "Panier moyen \x80 HT (Taux budget)", "Indice de vente"]

Afficher un comptage différentes modalités des variables catégoriques. Qu'en déduisez-vous ?

In [ ]:
for cat in categorical_variables: 
    print("Variable : {}".format(cat))
    #print(remplir ici)
    print()

Afficher quelques statistiques sur les variables numériques

In [ ]:
for num in numerical_variables : 
    print("Variable : {}".format(num))
    #print(remplir ici)
    print()

Données manquantes : 

Afficher les données manquantes pour chaque variable.
Quelles données peut-on compléter ? Quels choix faites vous en terme de gestion des données manquantes et pourquoi ?

In [ ]:
#Afficher les données manquantes ici

In [ ]:
mag.isnull().sum()

In [ ]:
mag["Pays magasin"].value_counts()

In [ ]:
mag["Mall"] = mag["Mall"].fillna("NON")

Pour des raisons de simplicité, nous enlevons les variables contenant plus de 20 % de données manquantes. Vous êtes bien entendu libres de tester différentes imputations de données, comme imputer avec la valeur la plus présente par exemple.

Nous pouvons également certaines colonnes jugées inutiles (Réseau, Code Magasin). La colonne Pays Magasin peut être retravaillée si vous le souhaitez. Nous l'enlevons ici.

In [ ]:
mag = mag.drop(["Réseau", "Pays magasin", "Code magasin", 
                'Environnement concurrentiel', 'Traffic', 'Clientèle', 'Saisonnalité',
       'Géographie'], axis = 1)

#### Fichier ventes

*Insérer vos conclusions ici.*

Combien y'a-t-il de magasins dans ce fichier, combien de y'a-t-il de produits ? Comparer le nombre de magasins ici avec le fichier magasins.

In [ ]:
# Explorer le dataset

In [ ]:
categorical_values_ventes = ["Collection produit", 'Classification famille',
       'Classification rayon', 'Classification sous famille',
       'Statut Collection Produit']
numerical_values_ventes = ['Ventes_17_19']

Afficher un comptage différentes modalités des variables catégoriques. Qu'en déduisez-vous ?

In [ ]:
for cat in categorical_values_ventes :
    print("Variable : {}".format(cat))
    #print(put here)
    print()

#### Fichier de références

*Insérez vos conclusions ici.*

- Le fichier de références contient des informations sur 288 produits, et 357 lignes contenant avec le détail du coloris.
- Il convient de faire une jointure entre le fichier des références et des ventes. Pour ceci, il faut créer la clé de jointure :
    - Pour la table ventes, il s'agit du champ Modèle_Coloris, qui est une concaténation des champs "Référence produit" et "Couleur produit"
    - Pour la table références, il faut créer la clé de jointure en concaténant le champ "Produit" et le champ "Libellé couleurs"

In [ ]:
# Exploration de jeu de données

In [ ]:
# Exploration des variables catégoriques et numériques

In [ ]:
#Création  de la clé de jointure
ref["Modèle_Coloris"] = ref["Produit"] + ref["Libellé couleurs"]

### I.1.2. Jointure entre le fichier de ventes et de références

Nous effectuons une jointure "inner" entre les ventes et les références. L'objectif est de ne garder que les informations contenues dans les deux fichiers.

Nous calculons ensuite le montant total pour chaque référence produit et chaque magasin.

In [ ]:
ventes_ref = ventes.merge(ref, how = 'inner', on = "Modèle_Coloris")

In [ ]:
# Calculer le Montant Total
ventes_ref["Montant total"] = ventes_ref["Ventes17_19"] * ventes_ref["PVB"]

### I.1.3. Création de variables à la maille du magasin sur le fichier joint des ventes et des références

Etant donné que nous souhaitons faire un clustering de magasins, il convient de calculer de nouvelles features à partir du dataset "ventes_ref" à la maille du magasin.
La fonctionnalité groupby permet de répondre à ce besoin.


**Calcul du montant total et du nombre de ventes total par magasin**

In [ ]:
mag_montants_ventes = ventes_ref.groupby("Magasin")[["Montant total", "Ventes17_19"]].sum()

Calculez le prix de vente moyen

In [ ]:
#mag_montants_ventes["Prix de vente moyen"] = 

**Calcul du pourcentage de ventes des produits iconiques par magasin** 

In [ ]:
ventes_ref["Iconique"].value_counts(dropna = False)

In [ ]:
#On remplace les champs Oui et Non par "Iconique" et "Non_iconique" pour pouvoir les reconnaitre plus tard

In [ ]:
ventes_ref["Iconique_transformed"] = ventes_ref["Iconique"].map({"OUI":"Iconique", "NON": "Non_iconique"})

In [ ]:
iconique_sum = ventes_ref.groupby(["Magasin", "Iconique_transformed"])['Ventes17_19'].sum().unstack()

In [ ]:
iconique_sum.head()

Nous remarquons certaines valeurs manquantes. Cela veut dire que dans certains magasins (en l'occurence, les magasins 146 et 89), aucun produit iconique n'a été vendu. Nous allons donc imputer les valeurs manquantes à 0.

In [ ]:
iconique_sum[iconique_sum.Iconique.isnull()]

In [ ]:
iconique_sum = iconique_sum.fillna(0)

In [ ]:
iconique_prc = iconique_sum.div(iconique_sum.sum(axis = 1), axis = 0)

In [ ]:
iconique_prc.head()

**Faire le même traitement pour d'autres colonnes (nous proposon ces colonnes, mais il est possible d'en rajouter d'autres si vous les trouvez pertinentes) :**
- Prix (selon famille)
- Type saison
- Segment
- Libellé thème
- Statut de collection

Il peut être pertinent de créer une boucle for et de stocker les datasets temporaires dans une liste, puis de les concaténer

In [ ]:
#On remplace les champs Oui et Non par "Iconique" et "Non_iconique" pour pouvoir les reconnaitre plus tard
ventes_ref["Iconique_transformed"] = ventes_ref["Iconique"].map({"OUI":"Iconique", "NON": "Non_iconique"})

#On rajoute le nom de la feature pour reconnaitre les catégories
ventes_ref["Prix (selon famille)_transformed"] = "Famille prix_"+ventes_ref["Prix (selon famille)"]

#Même chose pour le type de saison
ventes_ref["Type saison_transformed"] = "Type saison_"+ventes_ref["Type saison"]

#Même chose pour le Segment
ventes_ref["Segment_transformed"] = "Segment_"+ventes_ref["Type saison"]

#Même chose pour le Libellé thème
ventes_ref["Libellé thème_transformed"] = "Libellé thème_"+ventes_ref["Libellé thème"]

#Même chose pour le Libellé thème
ventes_ref["Statut Collection Produit_transformed"] = "Statut Collection Produit_"+ventes_ref["Libellé thème"]

Réalisez les mêmes traitements que pour le champ "Iconique"

In [ ]:
# features_magasin = mettre ici toutes vos features crées à la maille du Magasin

**Nous vous encourageons à rajouter de nouvelles variables si elles vous paraissent pertinentes.**

### I.1.4. Concaténation avec le fichier des magasins

In [ ]:
mags_final = mag.merge(features_magasins, left_on = "Magasin", right_index = True )

In [ ]:
# nous définissons la colonne Magasin comme étant l'index de chaque ligne
mags_final = mags_final.set_index('Magasin')

In [ ]:
mags_df = mags_final[cols_num].copy()

# II. Analyse en composantes principales

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import numpy as np

Comme expliqué en cours, l'analyse en composantes principales (PCA) est une méthode factorielle de réduction de dimension pour l’exploration de données quantitatives. Elle permet de transformer des variables corrélées en nouvelles variables décorrélées, appelées composantes principales.

In [ ]:
# Réaliser une PCA Ici (pensez à tester avec des données normalisées)


In [ ]:
# Explained variance ratio des composantes principales (qu'on pourrait comparer avec celles des features originales...)


Représentez des magasins sur les composantes principales

Analysez les composantes principales comme vu en cours : Quelles sont les variables les plus corrélées avec les composantes principales ? 

# III. Clustering

In [ ]:
from sklearn.cluster import KMeans

## II.1. Méthode du coude pour définir le nombre de clusters

Il peut être intéressant de tester le clustering avec les données brutes, les données normalisées, et les données transformées avec l'ACP.

In [ ]:
#Méthode du coude

In [ ]:
# Entrainer un k-means ici


In [ ]:
# Ajoutez les labels des clusters au dataframe

### Caractéristisation des clusters :

Analysez chacun des clusters : Quelle est la distribution des variables ? Nous vous invitons à filtrer sur chacun des clusters et à établir des statistiques.

*Commencez avec des analyses statistiques simples*

Taille des clusters

In [ ]:
mags_df["Cluster"].value_counts()

In [ ]:
mags_df.groupby("Cluster").mean()

In [ ]:
mags_df.mean()

**Portez une attention particulière à l'analyse et l'interprétation des premiers résultats du clustering. Ces résultats pourront vous inspirer la création de nouvelles variables plus pertinentes, ou l'adoption d'une nouvelle démarche. Par exemple, si on trouve un cluster principal qui contient la majorité des magasins et d'autres clusters avec un très faible nombre de magasins, il peut être intéressant de les écarter et de refaire un clustering uniquement sur le gros cluster.**

**On tombe rarement sur la bonne réponse du premier coup, c'est une approche très itérative.**
